# 1.以Dialogflow實現多輪對話

### (請參考slide"Google Dialogflow 簡介"完成實作)

# 2.Linebot設計

### (請參考slide"Linebot 基本設定"完成Linebot串接)

In [2]:
from __future__ import unicode_literals
import os, requests, json, configparser
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *

app = Flask(__name__)

# LINE 聊天機器人的基本資料
config = configparser.ConfigParser()
config.read('config.ini')

line_bot_api = LineBotApi(config.get('line-bot', 'channel_access_token'))
handler = WebhookHandler(config.get('line-bot', 'channel_secret'))

# 接收 LINE 資訊
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        print("body:",body)
        print("signature:", signature)
        print("===")
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 回覆設定 (defaul僅能回固定語句)
@handler.add(MessageEvent, message=TextMessage)
def get_response(event):
    query = event.message.text   
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text="不論你說什麼 我都回你好!"))

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Apr/2021 17:17:12] "POST / HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2021 17:17:24] "POST /callback HTTP/1.1" 200 -


body: {"events":[],"destination":"U2f7ba918a6523b0e2acd7dee8833d56b"}
signature: h2PyxqCW5PlfWdqTmPeOCBd5gnQ+zVa1Q5PtsC0Z3MI=
===


127.0.0.1 - - [11/Apr/2021 17:17:54] "POST /callback HTTP/1.1" 200 -


body: {"events":[{"type":"follow","replyToken":"4ef3dda6d978424fa144961130e91b90","source":{"userId":"Ub48cc7d7b61198bd5271236b99d2c30f","type":"user"},"timestamp":1618132672685,"mode":"active"}],"destination":"U2f7ba918a6523b0e2acd7dee8833d56b"}
signature: 73ASJMWc8LBJrOpzcvCTTbQ5tmEWUZRIdXtSR6diAAQ=
===
body: {"events":[{"type":"message","replyToken":"de51dffcab2e424da358bd4fe5d9f581","source":{"userId":"Ub48cc7d7b61198bd5271236b99d2c30f","type":"user"},"timestamp":1618132679546,"mode":"active","message":{"type":"text","id":"13871324667076","text":"hi"}}],"destination":"U2f7ba918a6523b0e2acd7dee8833d56b"}
signature: imYoZBf6HMqYpd1uaxWOl8pD0S515nXeLndVm+IHNtw=
===


...

body: {"events":[{"type":"message","replyToken":"13aab5be4cee4d5e993cdfc15346d05f","source":{"userId":"Ub48cc7d7b61198bd5271236b99d2c30f","type":"user"},"timestamp":1618132685526,"mode":"active","message":{"type":"text","id":"13871325059738","text":"？"}}],"destination":"U2f7ba918a6523b0e2acd7dee8833d56b"}
signature: oZa2L2XbA1pn+al4usf7k9MCaL3i66RTSHmVr84AM14=
===


127.0.0.1 - - [11/Apr/2021 17:18:41] "POST /callback HTTP/1.1" 200 -


body: {"events":[{"type":"message","replyToken":"23d5ea8e19c8449f956c5fb0be80335f","source":{"userId":"Ub48cc7d7b61198bd5271236b99d2c30f","type":"user"},"timestamp":1618132719965,"mode":"active","message":{"type":"text","id":"13871327325085","text":"hi"}}],"destination":"U2f7ba918a6523b0e2acd7dee8833d56b"}
signature: swuSdTVOCUjLNp75MYjNBPScZG+KKIcxa370wd83jBg=
===


127.0.0.1 - - [11/Apr/2021 17:18:47] "POST /callback HTTP/1.1" 200 -


body: {"events":[{"type":"message","replyToken":"57e3904dbded4dd198b7443d537caff0","source":{"userId":"Ub48cc7d7b61198bd5271236b99d2c30f","type":"user"},"timestamp":1618132725863,"mode":"active","message":{"type":"text","id":"13871327717203","text":"為什麼"}}],"destination":"U2f7ba918a6523b0e2acd7dee8833d56b"}
signature: q58Qcn7CcJyMiINxiFnqC23PToK53BcZvsNePQP4kVo=
===


# 3.以Python實現多輪對話

## 3-1.建立高鐵API函式

### (請參考 thsr_utils.py 完成函式)

In [ ]:
# 其他API呼叫方式請參考 => https://ptx.transportdata.tw/MOTC/?t=Rail&v=2

## 3-2.建立多輪判斷邏輯

In [6]:
from __future__ import unicode_literals
import os, requests, json, configparser
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *

import stock_utils as Stock
import thsr_utils as Thsr
thsr = Thsr.ThsrModule()

chat_record = []
thsr_res = {"starting":"", "ending":"", "date":"", "ampm":""}
station_names = Thsr.station_name
date_keys = Thsr.date_key
ampm_keys = Thsr.ampm_key


app = Flask(__name__)

# LINE 聊天機器人的基本資料
config = configparser.ConfigParser()
config.read('config.ini')

line_bot_api = LineBotApi(config.get('line-bot', 'channel_access_token'))
handler = WebhookHandler(config.get('line-bot', 'channel_secret'))

# 接收 LINE 資訊
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        print("body:",body)
        print("signature:", signature)
        print("===")
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 回覆設定 (加入高鐵API多輪對話)
@handler.add(MessageEvent, message=TextMessage)
def get_response(event):
    query = event.message.text       
    
    if len(chat_record) < 5:
        chat_record.append(query)
    else:
        chat_record.pop(0)
        chat_record.append(query)
    print("chat_record:",chat_record)


    # 判斷是否為"高鐵查詢意圖"
    if query == "高鐵":
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="哪一天出發?"))
    try:
        if chat_record[-2] == "高鐵" and any(chat_record[-1] == i for i in date_keys):
            date_format = thsr.get_date_string_today(chat_record[-1])
            thsr_res['date'] = date_format
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text="上午還是下午的車?"))

        elif any(chat_record[-2] == i for i in date_keys) and any(chat_record[-1] == i for i in ampm_keys):
            thsr_res['ampm'] = chat_record[-1]
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text="起站是哪裡呢?"))

        elif any(chat_record[-2] == i for i in ampm_keys) and any(chat_record[-1] == i for i in station_names):
            startind_id = Thsr.station_id[chat_record[-1]]
            thsr_res['starting'] = startind_id
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text="終點站是哪裡呢?"))

        elif any(chat_record[-2] == i for i in station_names) and any(chat_record[-1] == i for i in station_names):
            ending_id = Thsr.station_id[chat_record[-1]]
            thsr_res['ending'] = ending_id

            # print("***",thsr_res['starting'],thsr_res['ending'],thsr_res['date'],thsr_res['ampm'])
            text = thsr.get_runs(thsr_res['starting'],thsr_res['ending'],thsr_res['date'],thsr_res['ampm'])
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text=text))

    except IndexError:
        pass
    
         
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text="不論你說什麼 我都回你好!"))

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Apr/2021 17:24:28] "POST /callback HTTP/1.1" 200 -


body: {"events":[{"type":"message","replyToken":"f050388082c2426fb2d29564ba660fb7","source":{"userId":"Ub48cc7d7b61198bd5271236b99d2c30f","type":"user"},"timestamp":1618133067135,"mode":"active","message":{"type":"text","id":"13871350257761","text":"要"}}],"destination":"U2f7ba918a6523b0e2acd7dee8833d56b"}
signature: tVJBoAAhQ7RQz86s+eg025zXm5C3RQpZBKbZ3GydVG8=
===
chat_record: ['要']
body: {"events":[{"type":"message","replyToken":"097d6cbe70b44761acecc61488b4e420","source":{"userId":"Ub48cc7d7b61198bd5271236b99d2c30f","type":"user"},"timestamp":1618133071723,"mode":"active","message":{"type":"text","id":"13871350553707","text":"高鐵"}}],"destination":"U2f7ba918a6523b0e2acd7dee8833d56b"}
signature: EHrnRGFQ//Ou6XS7do2BQ7VCqviIC73PbLLJ7J2w9Xw=
===
chat_record: ['要', '高鐵']


[2021-04-11 17:24:33,536] ERROR in app: Exception on /callback [POST]
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-6-04d999ef4ece>", line 39, in callback
    handler.handle(

body: {"events":[{"type":"message","replyToken":"fe5654d6403548b2add2cdbf848160ab","source":{"userId":"Ub48cc7d7b61198bd5271236b99d2c30f","type":"user"},"timestamp":1618133076580,"mode":"active","message":{"type":"text","id":"13871350878588","text":"明天"}}],"destination":"U2f7ba918a6523b0e2acd7dee8833d56b"}
signature: Dk6QsniBuz7sGwiq7JBSCWu0OcsIs+3E6kugSRUP41s=
===
chat_record: ['要', '高鐵', '明天']


...

[2021-04-11 17:25:10,523] ERROR in app: Exception on /callback [POST]
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-6-04d999ef4ece>", line 39, in callback
    handler.handle(

body: {"events":[{"type":"message","replyToken":"54116959c02745f291f39f6c9c8debd6","source":{"userId":"Ub48cc7d7b61198bd5271236b99d2c30f","type":"user"},"timestamp":1618133112350,"mode":"active","message":{"type":"text","id":"13871353254127","text":"板橋"}}],"destination":"U2f7ba918a6523b0e2acd7dee8833d56b"}
signature: LTcQDaRf05Mqnkyu4o27Kyg1SeruKarLwdpeNXgxKZU=
===
chat_record: ['板橋', '高鐵', '明天', '下午', '板橋']


[2021-04-11 17:25:13,314] ERROR in app: Exception on /callback [POST]
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-6-04d999ef4ece>", line 39, in callback
    handler.handle(

body: {"events":[{"type":"message","replyToken":"3adfa7a48a1540dfa5e86da29527d2dd","source":{"userId":"Ub48cc7d7b61198bd5271236b99d2c30f","type":"user"},"timestamp":1618133114712,"mode":"active","message":{"type":"text","id":"13871353411117","text":"苗栗"}}],"destination":"U2f7ba918a6523b0e2acd7dee8833d56b"}
signature: mdMAGTtiLIh0OBMg9Die/lPAviloIkzHc8wnE4YU8Wk=
===
chat_record: ['高鐵', '明天', '下午', '板橋', '苗栗']


[2021-04-11 17:25:16,286] ERROR in app: Exception on /callback [POST]
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-6-04d999ef4ece>", line 39, in callback
    handler.handle(